In [1]:
from pymongo import MongoClient
client = MongoClient()
entries = client['usta']['teams']

In [5]:
from pandas import DataFrame
from pandas import ExcelWriter

In [2]:
entries.count()

/Users/patrick/miniconda3/envs/usta/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: count is deprecated. Use estimated_document_count or count_documents instead. Please note that $where must be replaced by $expr, $near must be replaced by $geoWithin with $center, and $nearSphere must be replaced by $geoWithin with $centerSphere
  """Entry point for launching an IPython kernel.


26822

In [3]:
entries.distinct("season_name", {"year": "2019"})

['2019 Adult 18 & Over',
 '2019 Adult 18 - 45 Team Singles League',
 '2019 Adult 18-39 League',
 '2019 Adult 40 & Over',
 '2019 Adult 55 & Over',
 '2019 Adult 65 & Over',
 '2019 Adult 70 & Over',
 '2019 Mixed 18 & Over',
 '2019 Mixed 40 & Over',
 '2019 Mixed 55 & Over']

In [25]:
columns = ['year', 'season', 'captains', 'teams', 'organizations']
data = {k: [] for k in columns}
years = map(str, entries.distinct("year"))
seasons = [f"{comp} {age} & Over" for comp in ["Adult", "Mixed"] for age in [18, 40]]
for year in years:
    for season in seasons:
        q = {"year": year, "season_name": f"{year} {season}"}
        data['year'].append(year)
        data['season'].append(season)
        data['captains'].append(len(entries.distinct('captain', q)))
        data['teams'].append(entries.count(q))
        data['organizations'].append(len(entries.distinct('organization', q)))
        
df = DataFrame(data=data)
#df.set_index(['year', 'season'], inplace=True)

/Users/patrick/miniconda3/envs/usta/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning:

count is deprecated. Use estimated_document_count or count_documents instead. Please note that $where must be replaced by $expr, $near must be replaced by $geoWithin with $center, and $nearSphere must be replaced by $geoWithin with $centerSphere



In [26]:
df

,year,season,captains,teams,organizations
0,2013,Adult 18 & Over,1130,1217,239
1,2013,Adult 40 & Over,896,952,212
2,2013,Mixed 18 & Over,460,506,176
3,2013,Mixed 40 & Over,312,330,145
4,2014,Adult 18 & Over,1102,1195,228
5,2014,Adult 40 & Over,843,906,205
6,2014,Mixed 18 & Over,0,0,0
7,2014,Mixed 40 & Over,0,0,0
8,2015,Adult 18 & Over,1106,1211,228
9,2015,Adult 40 & Over,958,1020,222


In [11]:
import plotly.express as px

In [38]:
df = df[df["captains"] > 0]
for col in ["captains", "teams", "organizations"]:
    px.line(
        df,
        x='year', 
        y=col,
        color="season"
    ).show()

In [39]:
def save_xls(dct, xls_path='usta_teams.xlsx'):
    writer = ExcelWriter(xls_path)
    for name, s in dct.items():
        s.to_excel(writer, sheet_name=name)
    writer.save()

In [40]:
seasons = entries.distinct("season_name")
len(seasons)

66

In [41]:
skip = ['_id', 'id', 'league', 'season_id', 'season_name']
projection = {k: 0 for k in skip}
columns = list(entries.find_one({}, projection).keys())
columns

['year',
 'league_url',
 'gender',
 'level',
 'team',
 'team_url',
 'captain',
 'captain_url',
 'city',
 'area',
 'organization',
 'organization_url',
 'created']

In [43]:
sheets = {}

for season in reversed(seasons):
    docs = entries.find({"season_name": season}, projection)
    print(season, docs.count())
    data = {c: [] for c in columns} 
    for doc in docs:
        for k, v in doc.items():
            data[k].append(v)

    season = season.replace('/', ' ')[:32]
    sheets[season] = DataFrame(data=data)
    sheets[season].set_index('team', inplace=True)
    sheets[season].sort_values(by=['year', 'gender', 'level', 'team'], inplace=True)

save_xls(sheets)
print('DONE')

/Users/patrick/miniconda3/envs/usta/lib/python3.7/site-packages/ipykernel_launcher.py:5: DeprecationWarning:

count is deprecated. Use Collection.count_documents instead.



2020 Reno/Tahoe Adult 40 29
2020 Mixed 55 & Over 140
2020 Mixed 40 & Over 276
2020 Adult 65 & Over 117
2020 Adult 40 & Over 815
2020 Adult 18 & Over 936
2019 Mixed 55 & Over 149
2019 Mixed 40 & Over 316
2019 Mixed 18 & Over 385
2019 Adult 70 & Over 33
2019 Adult 65 & Over 111
2019 Adult 55 & Over 338
2019 Adult 40 & Over 822
2019 Adult 18-39 League 33
2019 Adult 18 - 45 Team Singles League 88
2019 Adult 18 & Over 984
2018 Mixed 55 & Over 157
2018 Mixed 40 & Over 311
2018 Mixed 18 & Over 434
2018 Adult 70 & Over 40
2018 Adult 65 & Over 122
2018 Adult 55 & Over 350
2018 Adult 40 & Over 809
2018 Adult 18 - 39 League 54
2018 Adult 18 & Over 1025
2018 18-39 Team Singles League 67
2017 Mixed 40 & Over 320
2017 Mixed 18 & Over 430
2017 Mens 2.5+ Pilot 6
2017 Adult 70 & Over 42
2017 Adult 65 & Over 122
2017 Adult 55 & Over 365
2017 Adult 40 & Over 1043
2017 Adult 18 & Over 1142
2016 Mixed 55 & Over 141
2016 Mixed 40 & Over 341
2016 Mixed 18 & Over 470
2016 Adult 70 & Over 33
2016 Adult 65 & Ov